In [0]:
import requests
from pyspark.sql.functions import collect_list

In [0]:
def send_message(message, chat_ids):
    TOKEN = dbutils.secrets.get("job_notification", "Telegram_Token")
    CHAT_ID = chat_ids
    MESSAGE = message

    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage"
    for user in CHAT_ID:
        payload = {
            "chat_id": user,
            "text": MESSAGE,
            "parse_mode": "HTML"
        }

        res = requests.post(url, data=payload)

In [0]:
df = spark.table('main_catalogue.jobs.users')
chat_ids = df.filter(df.skill == "Data Engineer").select(collect_list('chat_id')).collect()[0][0]

In [0]:
linkedin_jobs = spark.table("staging_catalogue.jobs.linkedJobs") \
                .select('url', 'title', 'company', 'location', 'hrs_ago', 'score')
naukari_jobs = spark.table("staging_catalogue.jobs.naukariJobs") \
                .select('url', 'title', 'companyName', 'location', 'hrs_ago', 'score')
indeed_jobs = spark.table("staging_catalogue.jobs.indeedJobs") \
                .select('url', 'title', 'company', 'location', 'hrs_ago', 'score')

In [0]:
combined_df = linkedin_jobs.union(naukari_jobs) \
                            .union(indeed_jobs)
combined_df = combined_df.orderBy(combined_df["score"].desc())
combined_df.write.mode('overwrite').option("mergeSchema", "true").saveAsTable("main_catalogue.jobs.curated_jobs")
display(combined_df)

In [0]:
s = ''
count = 1
tc = 1
tl = combined_df.count()
for row in combined_df.collect():
    # print(row)
    s += f"""{tc}. <a href = "{row["url"]}" >{row["title"]}</a>\n<b>{row["company"]}</b>, <i>{row["location"]}, {row["hrs_ago"]}</i> \n\n"""
    if count==20 or tc==tl:
        send_message(s, chat_ids)
        print(s)
        count = 0
        s = ''
    count+=1
    tc+=1